# Demo: mrc2png
Convert raw MRC files to PNG with various pre-processing options including adaptive histogram equalisation.

In [ ]:
import os
import sys
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# Automatically reload imported programmes
%load_ext autoreload
%autoreload 2

# Directories (ammend as necessary)
root_dir = '/content/gdrive/MyDrive/IDSAI/PROOF/filament-segmentation'
os.chdir(root_dir)  # Move to root_dir
sys.path.insert(0, root_dir)

In [ ]:
# GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
   print(gpu_info)

In [ ]:
%%script false
!pip install mrcfile

In [ ]:
import argparse
from glob import glob
import mrcfile
import numpy as np
import os
import skimage
from skimage import exposure
from skimage import io
from skimage import transform
from tqdm import tqdm

In [ ]:
# Pass options for MRC to PNG conversion
class Options:
    def __init__(self):
        self.mrc_dir = '/content/drive/MyDrive/Colab/tomograms/original_mrc'
        self.png_dir = None
        self.printout = False
        self.ahe = False
        self.clip = True
        self.keep_dims = True
        self.width = None
        self.height = None
        self.depth = None

args=Options()
if args.keep_dims:
    (args.depth, args.height, args.width) = (None, None, None)    

In [ ]:
# Check directories exists.
if not os.path.isdir(args.mrc_dir):
    raise IOError("Directory doesn't exist: {}".format(args.mrc_dir))
if not args.png_dir:
    parent_dir = os.path.abspath(os.path.join(args.mrc_dir, os.pardir))
    args.png_dir = parent_dir + '/processed_png'
    print(args.png_dir)
if not os.path.isdir(args.png_dir):
    os.makedirs(args.png_dir)
    print('Made new PNG dir: ', args.png_dir)

In [ ]:
# List all MRC files in the directory.
mrcs = glob(args.mrc_dir + '/*.mrc')
if not mrcs:
    print('No MRC files here: ' + args.mrc_dir)

In [ ]:
# Iterate conversion and processing routine
for idx, file in enumerate(mrcs):
    filename = os.path.basename(file)

    print("\nProcessing MRC #{num}: `{name}`".format(num=idx+1, name=filename))
    with mrcfile.open(file, permissive=True) as mrc:
        image = mrc.data
        if args.printout:
            mrc.print_header()

    print("Input image shape: ", image.shape)

    # Clip the 5% and 95% quartiles
    if args.clip:
        image = np.clip(
            image, np.percentile(image, 5), np.percentile(image, 95),
        )

    # Normalise image values over [0, 1]
    image = (image - image.min()) / (image.max() - image.min())

    # Resize output
    if any((args.depth, args.height, args.width)):
        print('Resizing image')
        output_shape = [args.height, args.width]
        if len(image.shape) == 3:
            output_shape = [args.depth] + output_shape
        for dim, length in enumerate(output_shape):
            if not length:
                output_shape[dim] = image.shape[dim]
        print("Output image shape: ", tuple(output_shape))
        image = transform.resize(image, output_shape, anti_aliasing=False)

    if args.ahe:
        print('Adaptive histogram equalisation')
        kernel_size = (image.shape[-2] // 5, image.shape[-1] // 5)
        if len(image.shape) == 3:
            kernel_size = (image.shape[0] // 2,) + kernel_size
        kernel_size = np.array(kernel_size)
        clip_limit = 0.9  # In [0, 1] where higher value increases contrast
        image = exposure.equalize_adapthist(image, kernel_size, clip_limit)
    else:
        print('Histogram equalisation')
        image = exposure.equalize_hist(image)

    # Renormalise to 8-bit unit for PNG save
    image = skimage.img_as_ubyte(image)

    print('Saving z-slices:')
    filename = filename.split('.')[0]
    if len(image.shape) == 2:
        io.imsave(args.png_dir + '/' + filename + '.png', image)
    elif len(image.shape) == 3:
        stackdir = args.png_dir + '/zstack_' + filename
        if not os.path.isdir(stackdir):
            os.makedirs(stackdir)
        for z in tqdm(range(image.shape[0])):
            io.imsave(stackdir + '/zslice{:04d}.png'.format(z), image[z])



Processing MRC #1: `tomo4.mrc`
Input image shape:  (294, 1855, 1919)
Histogram equalisation
Saving z-slices:


100%|██████████| 294/294 [10:53<00:00,  2.22s/it]



Processing MRC #2: `tomo1.mrc`
Input image shape:  (111, 1855, 1919)
Histogram equalisation
Saving z-slices:


100%|██████████| 111/111 [06:35<00:00,  3.56s/it]



Processing MRC #3: `tomo2.mrc`
Input image shape:  (156, 1855, 1919)
Histogram equalisation
Saving z-slices:


100%|██████████| 156/156 [08:49<00:00,  3.40s/it]



Processing MRC #4: `tomo3.mrc`
Input image shape:  (160, 1855, 1919)
Histogram equalisation
Saving z-slices:


100%|██████████| 160/160 [08:49<00:00,  3.31s/it]



Processing MRC #5: `tomo5.mrc`
Input image shape:  (211, 1855, 1919)
Histogram equalisation
Saving z-slices:


100%|██████████| 211/211 [03:41<00:00,  1.05s/it]
